# Данные

In [2]:
import csv
from collections import defaultdict, Counter
import random
import json
from metrics import apk
random.seed(42)

In [3]:
from sklearn.ensemble import *
import numpy as np

In [4]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

In [5]:
import lasagne
import theano
import theano.tensor as T

In [91]:
import tensorflow as tf

In [261]:
#Словари для основной выборки
user_to_items = defaultdict(set)
item_to_users = defaultdict(set)
edges = []
#Словари для тестовой выборки
test_user_to_items = defaultdict(set)
test_item_to_users = defaultdict(set)
test_edges = []

with open("data/train_likes.csv") as datafile:
    for like in csv.DictReader(datafile):
        # Кидаем монетку. В зависимости от результата кладём в обучение или тест
        if random.random() < 0.5:
            user_to_items[like['user_id']].add(like['item_id'])
            item_to_users[like['item_id']].add(like['user_id'])
            edges.append((like['user_id'], like['item_id']))
        else:
            test_user_to_items[like['user_id']].add(like['item_id'])
            test_item_to_users[like['item_id']].add(like['user_id'])
            test_edges.append((like['user_id'], like['item_id']))

In [262]:
all_items = set(item_to_users.keys()) | set(test_item_to_users.keys())
all_users = set(user_to_items.keys()) | set(test_user_to_items.keys())

In [263]:
users_list = list(user_to_items.keys())
test_users_list = list(test_user_to_items.keys())

In [264]:
user_to_i = {user: i for i, user in enumerate(all_users)}
item_to_i = {item: i for i, item in enumerate(all_items)}
all_users_list = list(all_users)
all_items_list = list(all_items)

In [265]:
matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
test_matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
for user, items in user_to_items.items():
    for item in items:
        matrix[user_to_i[user], item_to_i[item]] = True
for user, items in test_user_to_items.items():
    for item in items:
        test_matrix[user_to_i[user], item_to_i[item]] = True
matrix = matrix.tocsr()
test_matrix = test_matrix.tocsr()

In [266]:
matrix

<55863x23891 sparse matrix of type '<class 'numpy.float64'>'
	with 55163 stored elements in Compressed Sparse Row format>

In [267]:
u, s, vt = sp.sparse.linalg.svds(matrix.astype(np.float32), k=100)

In [268]:
u.shape, s.shape, vt.shape

((55863, 100), (100,), (100, 23891))

In [269]:
films = json.load(open('data/items.json'))
films = {a['id']:a for a in films}
for a in films.values():
    del a['id']

In [270]:
for f in films.values():
    if 'genre' in f:
        f[f['genre']] = 1
        del f['genre']

In [271]:
features = Counter()
for f in films.values():
    features.update(set(f.keys()))

In [272]:
features.most_common(20)

[('year', 138078),
 ('duration', 138078),
 (0, 34128),
 (3, 32154),
 ('f_114306', 30650),
 (1, 28635),
 ('f_79251', 16647),
 ('f_117573', 15295),
 ('f_138980', 15215),
 (5, 15008),
 ('f_64513', 14020),
 ('f_205162', 10693),
 ('f_122038', 10436),
 ('f_84602', 8377),
 ('f_68894', 8348),
 ('f_127793', 7946),
 ('f_72071', 7866),
 (4, 7386),
 ('f_210900', 7085),
 ('f_130202', 6371)]

In [273]:
small_features_size = 60 

In [274]:
small_features = set([_[0] for _ in features.most_common(small_features_size)])

In [275]:
feature_to_i = {feature: i for i, feature in enumerate(small_features)}

In [276]:
feature_to_i

{0: 0,
 1: 1,
 2: 6,
 3: 3,
 4: 4,
 5: 5,
 6: 8,
 'f_84602': 2,
 'f_127828': 10,
 'f_79251': 9,
 'f_83623': 12,
 'f_93782': 14,
 'f_151440': 15,
 'f_165994': 18,
 'f_46479': 17,
 'f_69077': 38,
 'f_64718': 28,
 'f_49558': 19,
 'f_93783': 20,
 'f_429': 13,
 'f_205164': 21,
 'f_110758': 22,
 'f_163241': 23,
 'f_187493': 24,
 'f_114306': 25,
 7: 7,
 'f_210900': 27,
 'f_253': 29,
 8: 11,
 'f_71149': 30,
 'f_64513': 26,
 'f_140034': 31,
 'f_72071': 32,
 'f_44763': 33,
 'f_173938': 40,
 'f_121936': 36,
 'f_109358': 37,
 'duration': 58,
 'f_127794': 39,
 'year': 41,
 'f_197169': 43,
 'f_122038': 44,
 'f_130202': 45,
 'f_138980': 46,
 'f_68894': 47,
 'f_117573': 48,
 'f_188358': 49,
 'f_55703': 50,
 'f_89959': 51,
 'f_154169': 52,
 'f_144994': 34,
 'f_71282': 42,
 'f_187216': 53,
 'f_49716': 35,
 'f_203955': 55,
 'f_127793': 56,
 'f_168177': 57,
 'f_44580': 54,
 'f_205162': 59,
 'f_99688': 16}

# Фильтрация пользователей
* Значительная часть пользователей имеет всего 1-2 просмотра. При всём желании, рекоммендовать им что-либо осмысленное при помощи рассматриваемого здесь метода мы вряд ли сможем. Для простоты вычислений, удалим их из выборки.
* Важно понимать, что качество на оставшихся пользователях скорее всего будет выше, чем на первоначальной выборке.

In [277]:
min_items_per_user = 2
from copy import copy
for user in copy(test_user_to_items).keys():
    
    n_items_per_user = len(user_to_items[user]) + len(test_user_to_items[user])
    
    if n_items_per_user <= min_items_per_user:
        del user_to_items[user]
        del test_user_to_items[user]

### Рекоммендующая функция
Позволим себе немного вольности: наша функция будет возвращать не вероятности, а список фильмов в порядке убывания "рекомендованности".

* Рекоммендованность фильма item пользователю user посчитаем так:
  * Для каждого фильма, полайканного пользователем user, найдём других людей, которым понравился фильм.
  * Сложим всех таких "друзей по лайкам" вместе и назовём соседями (__neighborhood__) пользователя.
  * Для фильма item узнаем его аудиторию - множество пользователей, которые его лайкнули
  * Пригодность фильма пользователю - то, насколько "друзьям по лайкам" пользователя нравится этот фильм.

Для примера, будем использовать косинусную меру расстояния
  
$ cos(u_{film}, u_{neighborhood}) = $ =$ u_{film} \cdot u_{neighborhood} \over |u_{film}| |u_{neighborhood}| $


$u_{neighborhood}$ зависит только от пользователя, но не от фильма, поэтому при сравнении фильмов по пригодности для одного пользователя, его можно исключить из формулы для простоты вычислений.

$ similarity(u_{film}, u_{neighborhood}) = $ $  u_{film} \cdot u_{neighborhood} \over |u_{film}| $
  
  
Распишем формулу подробно:

$ similarity(u_{film}, u_{neighborhood}) = $ $ \sum _{u_i} [u_i \in u_{film}] \cdot [u_i \in u_{neighborhood}] \over |u_{film}|  $

* u_i - очередной пользователь (в цикле по всем пользователям)
  
Выражение $[u_i \in u_{neighborhood}]$ здесь означает "сколько раз очередной пользователь входит в множество друзей по лайкам"
  
  

In [278]:
from math import sqrt
from collections import Counter

def recommend(user, n_best = 10):
    user_items = user_to_items[user]
    
    neighborhood = Counter()
    for item in user_items:
        neighborhood.update(item_to_users[item])
    
    #словарь {фильм -> пригодность фильма пользователю}
    item_similarities = {}
    
    for item in all_items:
        if item in user_items: continue
        item_users = item_to_users[item]
        if len(item_users) == 0: continue
        
        n_common_users = sum(neighborhood[user] for user in item_users)
        similarity = float(n_common_users) / sqrt(len(item_users))
        item_similarities[item] = similarity
    
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    
    return items_sorted[:n_best]

In [279]:
user_to_int = dict()
for i, user in enumerate(all_users):
    user_to_int[user] = i 

In [280]:
#Making dataset
user_features = defaultdict(lambda:defaultdict(lambda:0))
for user in list(user_to_items.keys())[0:]:
    for item in user_to_items[user]:
        if item in films:
            for feature, value in films[item].items():    
                user_features[user][feature]+=value
    for f in user_features[user]:
        user_features[user][f]/=len(user_to_items)

In [281]:
edges[0] in test_edges

False

In [282]:
features_size = 2*small_features_size + 1

In [283]:
def extract_features(user=None, item=None, X_sample = np.zeros(features_size, dtype='float32')):
    if user is not None:
        for f in user_features[user].keys()&small_features:
            X_sample[feature_to_i[f]] = user_features[user][f]
        if item is not None:
            X_sample[2*small_features_size+0] = user_film_mk2(user, item)
    if item is not None and item in films:
        for f in films[item].keys()&small_features:
            X_sample[small_features_size+feature_to_i[f]] = films[item][f]
    
    return X_sample

In [284]:
len(edges)

55344

In [285]:
def generate_random_samples(X_size = 100, use_test=False):
    X = np.zeros((X_size, features_size), dtype='float32')
    Y = np.zeros(X_size, dtype='int8')
    if use_test:
        local_users_list = test_users_list
        local_user_to_items = test_user_to_items
    else:
        local_users_list = users_list
        local_user_to_items = user_to_items
    for i in range(X_size):
        film =''
        while film not in films: 
            result = random.random()
            if result > 0.50:
                result = 1
            else:
                result = 0
            user = random.choice(local_users_list)
            if result==0:
                film = random.choice(all_items_list)
                if film in local_user_to_items[user]:
                    result = 1
            else:
                film = random.choice(list(local_user_to_items[user]|{''}))
        X[i] = extract_features(user, film)
        Y[i] = result
    return X, Y

In [286]:
def generate_samples(X_true = 100,  use_test=False):
    X_fake=X_true
    X = np.zeros((X_true+X_fake, features_size), dtype='float32')
    Y = np.zeros(X_true+X_fake, dtype='float32')
    if use_test:
        local_users_list = test_users_list
        local_user_to_items = test_user_to_items
        local_edges = test_edges 
    else: 
        local_users_list = users_list
        local_user_to_items = user_to_items
        local_edges = edges
    for i, (user, film)  in enumerate(local_edges[:X_true]):
        X[i] = extract_features(user, film)
        Y[i] = 1
    for i in range(X_true, X_true+X_fake):
        user = random.choice(local_users_list)
        film = random.choice(all_items_list)
        X[i] = extract_features(user, film)
        if film in local_user_to_items[user]:
            Y[i] = 1
    return X, Y

In [252]:
rf = RandomForestRegressor(n_estimators=500)

In [253]:
X, Y = generate_random_samples(100000)

In [254]:
rf.fit(X, Y)

KeyboardInterrupt: 

In [58]:
X, Y = generate_random_samples(100, use_test=False)
print("Train score: %s" %(rf.score(X, Y),))
X, Y = generate_random_samples(100, use_test=True)
print("Test score: %s" %(rf.score(X, Y),))

Train score: 0.878675890313
Test score: 0.202766900421


In [287]:
input_X = T.matrix("input X")
input_shape = [None, features_size]

target_y = T.matrix("target Y integer",dtype='float32')

In [296]:
#Input layer (auxilary)
layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

#fully connected layer, that takes input layer and applies 50 neurons to it.
# nonlinearity here is sigmoid as in logistic regression
# you can give a name to each layer (optional)
layer = lasagne.layers.BatchNormLayer(layer)

layer = lasagne.layers.DenseLayer(layer,num_units=50,
                                   nonlinearity = lasagne.nonlinearities.sigmoid,
                                   name = "hidden_dense_layer")
layer = lasagne.layers.DenseLayer(layer,num_units=50,
                                   nonlinearity = lasagne.nonlinearities.sigmoid,
                                   name = "hidden_dense_layer")
layer = lasagne.layers.DenseLayer(layer,num_units=20,
                                   nonlinearity = lasagne.nonlinearities.sigmoid,
                                   name = "hidden_dense_layer")

layer = lasagne.layers.DropoutLayer(layer, p=0.5)

dense_output = lasagne.layers.DenseLayer(layer,num_units = 1 ,
                                        nonlinearity = lasagne.nonlinearities.tanh,
                                        name='output', b=None)

In [297]:
#network prediction (theano-transformation)
y_predicted = lasagne.layers.get_output(dense_output)
#all network weights (shared variables)
all_weights = lasagne.layers.get_all_params(dense_output, trainable=True)

predict = theano.function([input_X], y_predicted)
print (all_weights)

[beta, gamma, hidden_dense_layer.W, hidden_dense_layer.b, hidden_dense_layer.W, hidden_dense_layer.b, hidden_dense_layer.W, hidden_dense_layer.b, output.W]


In [298]:
loss = lasagne.objectives.squared_error(y_predicted, target_y)
loss = lasagne.objectives.aggregate(loss, mode = 'mean')


updates_sgd = lasagne.updates.adadelta(loss, all_weights,learning_rate=0.01)

train_fun = theano.function([input_X,target_y],loss,updates= updates_sgd)


loss_fun = theano.function([input_X,target_y],loss)


In [ ]:
import time

num_epochs = 100 #amount of passes through the data

batch_size = 1000 #number of samples processed at each function call
num_batches = 300

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in (generate_samples(batch_size, use_test=False) for _ in range(num_batches)):
        inputs, targets = batch
        targets = targets.reshape((-1, 1))
        train_err_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_batches = 0
    for batch in (generate_samples(batch_size, use_test=True) for _ in range(num_batches)):
        inputs, targets = batch
        targets = targets.reshape((-1, 1))
        val_err += loss_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  train loss: %s" % (train_err / train_batches, ))
    print("  test loss: %s" % (val_err / train_batches, ))
    

Epoch 1 of 100 took 47.874s
  train loss: 0.248482666684
  test loss: 0.256554608581
Epoch 2 of 100 took 46.722s
  train loss: 0.247284901165
  test loss: 0.256372477171
Epoch 3 of 100 took 46.761s
  train loss: 0.245874528896
  test loss: 0.255463886883
Epoch 4 of 100 took 50.006s
  train loss: 0.243981953537
  test loss: 0.255082078194
Epoch 5 of 100 took 49.138s
  train loss: 0.24262902697
  test loss: 0.254432885154
Epoch 6 of 100 took 48.138s
  train loss: 0.240568305502
  test loss: 0.254004824513
Epoch 7 of 100 took 47.468s
  train loss: 0.238661476063
  test loss: 0.252689206136
Epoch 8 of 100 took 49.617s
  train loss: 0.236162299211
  test loss: 0.251836812979
Epoch 9 of 100 took 47.268s
  train loss: 0.233667052396
  test loss: 0.250873459204
Epoch 10 of 100 took 48.685s
  train loss: 0.23065448692
  test loss: 0.249171837162
Epoch 11 of 100 took 47.167s
  train loss: 0.226591415263
  test loss: 0.248092779274
Epoch 12 of 100 took 51.655s
  train loss: 0.223122254222
  test 

In [228]:
X, Y = generate_random_samples(10, use_test=True)
predict(X), Y

(array([[ 0.99576976],
        [ 0.34379049],
        [ 0.45306789],
        [ 0.18065817],
        [ 0.87963364],
        [ 0.32685928],
        [ 0.30049887],
        [ 0.66587964],
        [ 0.94503155],
        [ 0.66958117]]), array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int8))

In [183]:
learning_rate = 0.01
training_epochs = 200
batch_size = 1000
total_batch = 200
display_step = 1

In [156]:
x = tf.placeholder(tf.float32, [None, features_size]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 1]) 

W = tf.Variable(tf.zeros([features_size, 1]))
b = tf.Variable(tf.zeros([1]))

pred = tf.nn.tanh(tf.matmul(x, W) + b)

cost = tf.reduce_mean(tf.squared_difference(y, pred))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [157]:
sess = tf.Session()
sess.run(init)

In [163]:
for epoch in range(training_epochs):
    avg_cost = 0.

    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = generate_random_samples(batch_size)
        batch_ys = batch_ys.reshape((-1, 1))
        # Fit training using batch data
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                      y: batch_ys})
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

print ("Optimization Finished!")

# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy for 3000 examples
x_test, y_test =   generate_random_samples(batch_size, use_test=True)
y_test = y_test.reshape((-1, 1))
print ("Accuracy:", cost.eval({x: x_test, y: y_test}, session=sess))

Epoch: 0001 cost= 0.197359944
Epoch: 0002 cost= 0.196920267
Epoch: 0003 cost= 0.196819510
Epoch: 0004 cost= 0.196147946
Epoch: 0005 cost= 0.196093600
Epoch: 0006 cost= 0.195149677
Epoch: 0007 cost= 0.194767732
Epoch: 0008 cost= 0.194793865


KeyboardInterrupt: 

In [159]:
W.eval(session=sess)

array([[  6.39263499e-06],
       [  1.58444379e-06],
       [  1.72093326e-06],
       [  1.30214175e-06],
       [  5.04144646e-06],
       [  2.78425546e-06],
       [  1.67713119e-06],
       [  3.35766458e-06],
       [  1.04063997e-06],
       [  3.70196062e-06],
       [  1.53262081e-05],
       [  1.56174860e-06],
       [  1.44628314e-06],
       [ -2.96540602e-05],
       [  1.19626009e-06],
       [  1.27234807e-06],
       [  3.34417655e-06],
       [  9.86953410e-07],
       [  2.64779883e-05],
       [  5.46258798e-06],
       [  4.35487069e-02],
       [  4.35487069e-02],
       [  2.58548814e-03],
       [  4.35487069e-02],
       [ -8.12742580e-03],
       [  4.35487069e-02],
       [  3.76427569e-03],
       [  2.90592201e-03],
       [  4.35487069e-02],
       [  1.27365021e-03],
       [  4.00641002e-03],
       [  1.73721742e-02],
       [  1.39680263e-02],
       [  1.39039569e-02],
       [  1.31673804e-02],
       [  1.03020733e-02],
       [  2.82581579e-02],
 

(array([[ 0.10983541],
        [ 0.10352055],
        [ 0.20060187],
        [ 0.18288017],
        [ 0.04566551],
        [ 0.19504457],
        [ 0.16255407],
        [ 0.19504296],
        [ 0.00110639],
        [ 0.37141779]]), array([0, 0, 0, 1, 1, 0, 1, 0, 1, 0], dtype=int8))

In [161]:
X, Y = generate_random_samples(10, use_test=False)
sess.run([pred], feed_dict={x: X}), Y

([array([[ 0.40177947],
         [ 0.35771608],
         [ 0.35028359],
         [ 0.38270113],
         [ 0.34994081],
         [ 0.39418095],
         [ 0.34223077],
         [ 0.38585007],
         [ 0.42586172],
         [ 0.39199695]], dtype=float32)],
 array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1], dtype=int8))

In [332]:
X, Y = generate_random_samples(10, use_test=True)
X[0], rf.predict(X), Y

(array([  3.98906996e-05,   0.00000000e+00,   5.98360493e-05,
          0.00000000e+00,   0.00000000e+00,   3.98906996e-05,
          0.00000000e+00,   7.97813991e-05,   1.99453498e-05,
          9.97267489e-05,   3.98906996e-05,   3.98906996e-05,
         -3.40250190e-05,   1.99453498e-05,   1.99453498e-05,
          8.33385002e-06,   0.00000000e+00,   0.00000000e+00,
          1.99453498e-05,   1.99453498e-05,   1.00000000e+00,
          1.00000000e+00,   2.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          2.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   2.00000000e+00,  -6.53573573e-01,
          2.00000000e+00,   1.00000000e+00,   4.65432197e-01,
          1.00000000e+00,   1.00000000e+00,   2.00000000e+00,
          1.00000000e+00,   1.09823048e-03,   0.00000000e+00], dtype=float32),
 array([ 0.74074074,  0.09090909,  0.10869565,  0.06      ,  0.12727273,
         0.12962963,  0.11111111,  0.12121

In [34]:
def user_film_mk3(user, item, debug=False):
    X = np.zeros((1, features_size), dtype='float32')
    X[0] = extract_features(user, item)
    probs = rf.predict(X)
    if debug:
        print(probs)
    return probs[0]

In [35]:
def user_film_mk1(user, item, debug=False):
    #print(len(user_features[user]))
    cursum = 0
    curcnt = 0
    if item in films:
        for feature, value in films[item].items():
            if type(value) is int:
                curcnt+=1    
                cursum+=value*user_features[user][feature]
    if curcnt==0:
        curcnt+=1
    #cursum/=curcnt
    if debug:
        print(cursum)
    return cursum

In [36]:
def user_film_mk2(user, item, debug=False):
    ui = user_to_i[user]
    ii = item_to_i[item]
    if debug:
        print(ui, ii)
    a = np.dot(u[ui,:] * s, vt[:,ii])
    return a

In [285]:
user_film_mk3('26052b20aa96ed8d803dbfe4e9497192', '45ea2aa2143effcb6575daf0143e31b4', debug=True)

[ 0.]


0.0

In [321]:
def recommend_mk3_manual(user, n_best = 10, debug=False):
    user_items = user_to_items[user]

    item_similarities = {}
    cur_items = [item for item in all_items if item not in user_items and len(item_to_users[item])>0]
    
    X = np.zeros((len(cur_items), features_size), dtype='float32')
    X_user = extract_features(user)
    for i, item in enumerate(cur_items):
        X[i] = extract_features(user=None, item=item, X_sample=X_user)
    
    probs =rf.predict(X)
    item_similarities = {key: prob for key, prob in zip(cur_items, probs)}
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    if debug:
        print(X)
        for a in  items_sorted[:n_best]:
            print((a, item_similarities.get(a, 0)))
    return items_sorted[:n_best]

In [292]:
def recommend_mk4_manual(user, n_best = 10, debug=False):
    user_items = user_to_items[user]

    item_similarities = {}
    cur_items = [item for item in all_items if item not in user_items and len(item_to_users[item])>0]
    
    X = np.zeros((len(cur_items), features_size), dtype='float32')
    X_user = extract_features(user)
    for i, item in enumerate(cur_items):
        X[i] = extract_features(user=None, item=item, X_sample=X_user)
    
    probs =predict(X)
    item_similarities = {key: prob for key, prob in zip(cur_items, probs)}
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    if debug:
        print(X)
        for a in  items_sorted[:n_best]:
            print((a, item_similarities.get(a, 0)))
    return items_sorted[:n_best]

In [236]:
def generic_recommend(user_flim_function):
    def recommend(user, n_best = 10, debug=False):
        user_items = user_to_items[user]

        item_similarities = {}
        for item in all_items:
            if item in user_items: continue
            item_users = item_to_users[item]
            if len(item_users) == 0: continue

            item_similarities[item] = user_flim_function(user, item)

        items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
        if debug:
            for a in  items_sorted[:n_best]:
                print((a, item_similarities.get(a, 0)))
        return items_sorted[:n_best]
    return recommend

In [123]:
generic_recommend(user_film_mk2)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk2('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

('7682d6641caf183ffcf2616c9bb1e434', 0.08739379)
('b5e424608d041f38c2fd6125b3bcd40f', 0.062321573)
('006a602774c79d476cdbd5dff7dd24f1', 0.047887065)
('e9caaa12a277abf87ba07b84e09a4170', 0.045560129)
('aa5f2ca699da42e467e550f9f071fb3f', 0.044249389)
('bcee3a68dbfb6b4ea97ac53ceb1d3287', 0.044186898)
('e8047b4262e676d28f50816ac3fde1ca', 0.042720947)
('77c7998b3d3d41f3fc3aec786b4015ac', 0.038098648)
('1da6f84598fbeea050da49fe37297932', 0.037306551)
('2dc12b93072f94bca9438902bddc36c4', 0.035170436)
10976 651


0.0024645075

In [124]:
generic_recommend(user_film_mk1)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk1('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

('e13df82b3252eaeefc0c13599695f9c7', 0.000658196541476355)
('634e622e5c957c2cd8ddb98ca75ef937', 0.0005584697927678163)
('8faa6327a2ca2eb92e738ffc204d35c0', 0.0005584697927678163)
('64b6536649e20bd72d7183ec717b6c86', 0.0005385244430261086)
('5c6d1862dc1e82f340962b308999e603', 0.0005185790932844008)
('bcee3a68dbfb6b4ea97ac53ceb1d3287', 0.0005185790932844007)
('ef786a839c609561b76ce03768eace80', 0.0005185790932844007)
('ff987a60d68f5bbbefe97f8c5062711a', 0.0004986337435426931)
('753bce7ac492e6c95b07b85f8877cf5a', 0.0004786883938009853)
('7b0f1b0815292c0a550fabd5eeadc011', 0.00045874304405927767)
0


0

In [293]:
recommend_mk4_manual('a18f526db904f8f2ee4c8d178bfc818c', debug=True)

[[  3.42418862e-05   2.05451302e-04   6.84837723e-05 ...,   3.32924634e-01
    1.00000000e+00   1.80774500e-16]
 [  3.42418862e-05   2.05451302e-04   6.84837723e-05 ...,   3.32924634e-01
    1.00000000e+00   1.80774500e-16]
 [  3.42418862e-05   2.05451302e-04   6.84837723e-05 ...,   3.32924634e-01
    1.00000000e+00   1.80774500e-16]
 ..., 
 [  3.42418862e-05   2.05451302e-04   6.84837723e-05 ...,   4.15132821e-01
    1.00000000e+00   1.80774500e-16]
 [  3.42418862e-05   2.05451302e-04   6.84837723e-05 ...,   4.15132821e-01
    1.00000000e+00   1.80774500e-16]
 [  3.42418862e-05   2.05451302e-04   6.84837723e-05 ...,   2.63475370e+00
    1.00000000e+00   1.80774500e-16]]
('ab3279ea6dd62acd5a9c330a396c3b1a', array([ 1.]))
('634e622e5c957c2cd8ddb98ca75ef937', array([ 1.]))
('2c3754f00e6d27cb51d240ae4addd120', array([ 1.]))
('c3adb5ff8f46fcc5b2ab864c4de5023b', array([ 1.]))
('e9eefe65f665c414c361c226c0ca4641', array([ 1.]))
('6223bf138ae3bc3eeb93c8331e834df3', array([ 1.]))
('106756aba5d3

['ab3279ea6dd62acd5a9c330a396c3b1a',
 '634e622e5c957c2cd8ddb98ca75ef937',
 '2c3754f00e6d27cb51d240ae4addd120',
 'c3adb5ff8f46fcc5b2ab864c4de5023b',
 'e9eefe65f665c414c361c226c0ca4641',
 '6223bf138ae3bc3eeb93c8331e834df3',
 '106756aba5d378cf731f6683545a586c',
 'fa7a26762b9645be36a0913e73a7cd82',
 '15464106e8fe48bb01b9daef7696f52f',
 '9de1905753afad63dfcb7c33720e9c24']

In [238]:
list(test_user_to_items.items())[7]

('a18f526db904f8f2ee4c8d178bfc818c', {'7a040062b953a9a3e7fb1545def5773f'})

In [42]:
def recommend_dummy(user, n_best = 10):
    item_similarities = []
    for item in all_items:
        #пропустим те фильмы, которые пользователь уже просмотрел, если нас об этом попросили
        if item in user_to_items[user]: continue
        item_similarities.append(item)
         
    random.shuffle(item_similarities)
    #вернём n_best наиболее пригодных
    #print(items_sorted[:n_best])
    return item_similarities[:n_best]

# Оценка качества - map@k

In [45]:
check_quality(recommend_dummy,10,200)

0 / 200
100 / 200
AP@10 = 9.259259259259259e-05


In [324]:
check_quality(recommend_mk3_manual,10, 200)

0 / 200
100 / 200
AP@10 = 0.0


In [294]:
check_quality(recommend_mk4_manual,100, 200)

0 / 200
100 / 200
AP@100 = 1.1261261261261261e-05


In [237]:
check_quality(generic_recommend(user_film_mk1),100, 200)

0 / 200
100 / 200
AP@100 = 0.0014817629179331307


In [130]:
check_quality(generic_recommend(user_film_mk2),200,200)

0 / 200
100 / 200
AP@100 = 0.007631459343212682


In [132]:
check_quality(recommend,100, 200)

0 / 200
100 / 200
AP@100 = 0.003063415596733874


In [49]:
len(test_user_to_items)

2732

In [232]:
#сколько рекоммендаций рассматриваем
def check_quality(function, K = 10, max_n_users = len(test_user_to_items)):
    APatK_per_user = []
    user_list = list(test_user_to_items.keys())[:max_n_users]
    
    for i, user in enumerate(user_list):
        #фильмы, которые пользователю на самом деле нравятся
        test_items = test_user_to_items[user]

        #Выдать топ-K рекоммендаций
        recommendation_list = function(user,n_best=K)
        #Посчитать ap@k
        user_APatK = apk(test_items, recommendation_list,k=K)

        #и сложить в коробку
        APatK_per_user.append(user_APatK)

        #Progress bar
        if i % 100 ==0:
            print(i,'/',max_n_users)

        if i > max_n_users:
            break

    print('AP@{} = {}'.format(K, sum(APatK_per_user)/len(APatK_per_user)))


# Notes
* Кроме качества рекоммендаций, map@k ещё зависит от доли тестовой выборки, фильтрации и от самого K. Сравнивать качество разных алгоритмов имеет смысл только при одинаковом K и тестовой выборке.
* Давать полезные рекоммендации пользователям с малым числом просмотров тоже можно: например, можно выдавать наиболее популярные в целом фильмы.
* Разделение на обучение/тест честнее делать на по времени: первые 70% (например) лайков в обучение, остальные в тест. Это ближе к реальной жизни, когда вы сначала обучаете модель на логах, а потом применяете на новых сессиях пользователей.